In [2]:
import torch
torch.cuda.is_available()

True

In [3]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [4]:
import pandas as pd
import pandas.compat
import math
import numpy as np
from sklearn.metrics import classification_report
import torch.nn.functional as F

In [5]:
import torch
import os
from tqdm import tqdm,trange
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.model_selection import train_test_split

In [6]:
from transformers import (XLNetConfig, XLNetForSequenceClassification, XLNetTokenizer)

In [7]:
data_file_address = "data/actorsArtistAuthorPolitician_training_data.tsv"
sentences = []
labels = []
complete_data = []
with open(data_file_address) as data_file:
    for record in data_file.readlines():
        record_fields =record.strip().split("\t")
        if len(record_fields) != 2:
            continue
        complete_data.append([record_fields[0],record_fields[1]])
#        labels.append(record_fields[0])
#        sentences.append(record_fields[1])

In [10]:
import random
random.shuffle(complete_data)
percentage = 0.1 * len(complete_data) #10 percent
percentage_of_complete_data = []
for data in complete_data[0:percentage]:
    percentage_of_complete_data.append(data)
len(complete_data)

40000

In [9]:
sentences[0]
labels[0]

'1'

In [11]:
tag2idx={'0': 0, '1': 1, '2': 2, '3': 3}

In [12]:
# Mapping index to name
tag2name={tag2idx[key] : key for key in tag2idx.keys()}
tag2name

{0: '0', 1: '1', 2: '2', 3: '3'}

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()
n_gpu

8

In [14]:
torch.cuda.is_available()
device

device(type='cuda')

In [15]:
vocabulary = 'models/xlnet-base-cased/xlnet-base-cased-spiece.model'


In [16]:
# Len of the sentence must be the same as the training model
# See model's 'max_position_embeddings' = 512
max_len  = 64

In [17]:
# With cased model, set do_lower_case = False
tokenizer = XLNetTokenizer(vocab_file=vocabulary,do_lower_case=False)

In [18]:
max_len  = 64

full_input_ids = []
full_input_masks = []
full_segment_ids = []

SEG_ID_A   = 0
SEG_ID_B   = 1
SEG_ID_CLS = 2
SEG_ID_SEP = 3
SEG_ID_PAD = 4

UNK_ID = tokenizer.encode("<unk>")[0]
CLS_ID = tokenizer.encode("<cls>")[0]
SEP_ID = tokenizer.encode("<sep>")[0]
MASK_ID = tokenizer.encode("<mask>")[0]
EOD_ID = tokenizer.encode("<eod>")[0]

for i,sentence in enumerate(sentences):
    # Tokenize sentence to token id list
    tokens_a = tokenizer.encode(sentence)
    
    # Trim the len of text
    if(len(tokens_a)>max_len-2):
        tokens_a = tokens_a[:max_len-2]
        
        
    tokens = []
    segment_ids = []
    
    for token in tokens_a:
        tokens.append(token)
        segment_ids.append(SEG_ID_A)
        
    # Add <sep> token 
    tokens.append(SEP_ID)
    segment_ids.append(SEG_ID_A)
    
    
    # Add <cls> token
    tokens.append(CLS_ID)
    segment_ids.append(SEG_ID_CLS)
    
    input_ids = tokens
    
    # The mask has 0 for real tokens and 1 for padding tokens. Only real
    # tokens are attended to.
    input_mask = [0] * len(input_ids)

    # Zero-pad up to the sequence length at fornt
    if len(input_ids) < max_len:
        delta_len = max_len - len(input_ids)
        input_ids = [0] * delta_len + input_ids
        input_mask = [1] * delta_len + input_mask
        segment_ids = [SEG_ID_PAD] * delta_len + segment_ids

    assert len(input_ids) == max_len
    assert len(input_mask) == max_len
    assert len(segment_ids) == max_len
    
    full_input_ids.append(input_ids)
    full_input_masks.append(input_mask)
    full_segment_ids.append(segment_ids)
    
    if 3 > i:
        print("No.:%d"%(i))
        print("sentence: %s"%(sentence))
        print("input_ids:%s"%(input_ids))
        print("attention_masks:%s"%(input_mask))
        print("segment_ids:%s"%(segment_ids))
        print("\n")

No.:0
sentence: American hip hop artist who has gained fame for his numerous albums and mixtapes including The Chuuwee Channel, Paradiso, Sabbath, Club 27, Commerce EP, and more. American hip hop artist who has gained fame for his numerous albums and mixtapes including The Chuuwee Channel, Paradiso, Sabbath, Club 27, Commerce EP, and more. Dionte Chuuwee Hunter is an American hip hop artist from Sacramento, California. Chuuwee has established a strong underground presence and is regularly featured on popular hip hop blogs and websites, such as 2DopeBoyz, HipHopDX, DJBooth, AllHipHop, and others. He also received a feature in XXL magazine. He was interviewed by Forbes at SXSW '12 as a rising artist. The record producers Chuuwee has worked with in the past include Cookin' Soul, Lee Bannon, Large Professor, as well as many others. He is currently signed to Amalgam Digital and has released four free mixtapes through the label. He recently released a collaboration mixtape with Southern Cali

In [19]:
print(tag2idx)

{'0': 0, '1': 1, '2': 2, '3': 3}


In [20]:
# Make label into id
tags = [tag2idx[str(lab)] for lab in labels]
print(tags)

[1, 1, 1, 1, 2, 2, 0, 2, 2, 3, 2, 0, 2, 1, 0, 2, 3, 0, 0, 3, 3, 0, 0, 0, 3, 3, 2, 2, 2, 1, 0, 2, 0, 2, 2, 2, 1, 0, 0, 0, 0, 0, 1, 2, 0, 0, 1, 2, 0, 0, 2, 2, 1, 0, 0, 1, 0, 2, 3, 3, 1, 3, 3, 1, 1, 3, 0, 2, 3, 3, 0, 0, 1, 1, 0, 2, 3, 3, 2, 0, 2, 3, 2, 1, 2, 1, 1, 1, 1, 2, 3, 1, 1, 2, 0, 3, 2, 2, 0, 2, 2, 2, 1, 3, 3, 0, 1, 2, 2, 1, 1, 0, 2, 2, 1, 2, 2, 0, 2, 0, 3, 3, 2, 1, 1, 1, 0, 1, 0, 0, 3, 0, 0, 1, 0, 1, 2, 2, 1, 0, 2, 0, 3, 2, 3, 3, 1, 2, 1, 3, 2, 1, 0, 2, 0, 3, 1, 1, 0, 1, 1, 2, 1, 3, 3, 0, 1, 2, 2, 1, 1, 0, 1, 3, 2, 2, 0, 3, 0, 2, 0, 0, 3, 1, 1, 0, 1, 2, 3, 2, 0, 2, 2, 2, 0, 3, 1, 1, 1, 2, 3, 1, 1, 1, 1, 1, 3, 2, 0, 2, 1, 3, 3, 0, 1, 0, 3, 2, 2, 3, 3, 0, 1, 2, 3, 3, 1, 2, 3, 2, 1, 3, 1, 0, 2, 1, 1, 2, 0, 1, 1, 1, 0, 0, 2, 0, 0, 0, 2, 2, 1, 2, 3, 0, 1, 1, 2, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 2, 2, 1, 0, 2, 0, 0, 1, 2, 3, 1, 2, 0, 1, 0, 2, 3, 1, 1, 0, 2, 3, 2, 3, 1, 1, 2, 0, 1, 2, 0, 1, 1, 1, 0, 3, 3, 2, 3, 0, 1, 3, 0, 1, 0, 3, 2, 0, 0, 0, 2, 0, 1, 0, 3, 2, 0, 2, 3, 2, 0, 0, 0, 1, 1, 1, 

In [21]:
tr_inputs, val_inputs, tr_tags, val_tags,tr_masks, val_masks,tr_segs, val_segs = train_test_split(full_input_ids, tags,full_input_masks,full_segment_ids, 
                                                            random_state=4, test_size=0.3)

In [22]:
len(tr_inputs),len(val_inputs),len(tr_segs),len(val_segs)

(28000, 12000, 28000, 12000)

In [23]:
tr_inputs = torch.tensor(tr_inputs)
val_inputs = torch.tensor(val_inputs)
tr_tags = torch.tensor(tr_tags)
val_tags = torch.tensor(val_tags)
tr_masks = torch.tensor(tr_masks)
val_masks = torch.tensor(val_masks)
tr_segs = torch.tensor(tr_segs)
val_segs = torch.tensor(val_segs)

In [24]:
# Set batch num
batch_num = 32

In [25]:
# Set token embedding, attention embedding, segment embedding
train_data = TensorDataset(tr_inputs, tr_masks,tr_segs, tr_tags)
train_sampler = RandomSampler(train_data)
# Drop last can make batch training better for the last one
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_num,drop_last=True)

valid_data = TensorDataset(val_inputs, val_masks,val_segs, val_tags)
valid_sampler = SequentialSampler(valid_data)
valid_dataloader = DataLoader(valid_data, sampler=valid_sampler, batch_size=batch_num)

In [26]:
# In this document, contain confg(txt) and weight(bin) files
model_file_address = 'models/xlnet-base-cased'

In [27]:
# Will load config and weight with from_pretrained()
# Recommand download the model before using
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-pytorch_model.bin"
# Download model from "https://s3.amazonaws.com/models.huggingface.co/bert/xlnet-base-cased-config.json" 
model = XLNetForSequenceClassification.from_pretrained(model_file_address,num_labels=len(tag2idx))

In [28]:
model;

In [29]:
print(device)

cuda


In [30]:
# Set model to GPU,if you are using GPU machine
model.to(device);

In [31]:
# Add multi GPU support
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [32]:
# Set epoch and grad max num
epochs = 5
max_grad_norm = 1.0

In [33]:
# Cacluate train optimiazaion num
num_train_optimization_steps = int( math.ceil(len(tr_inputs) / batch_num) / 1) * epochs

In [34]:
# True: fine tuning all the layers 
# False: only fine tuning the classifier layers
# Since XLNet in 'pytorch_transformer' did not contian classifier layers
# FULL_FINETUNING = True need to set True
FULL_FINETUNING = True

In [35]:
if FULL_FINETUNING:
    # Fine tune model all layer parameters
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    # Only fine tune classifier parameters
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [36]:
# TRAIN loop
model.train();

In [37]:
print("***** Running training *****")
print("  Num examples = %d"%(len(tr_inputs)))
print("  Batch size = %d"%(batch_num))
print("  Num steps = %d"%(num_train_optimization_steps))
for _ in trange(epochs,desc="Epoch"):
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(train_dataloader):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_segs,b_labels = batch
        
        # forward pass
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        loss, logits = outputs[:2]
        if n_gpu>1:
            # When multi gpu, average it
            loss = loss.mean()
        
        # backward pass
        loss.backward()
        
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        
        # update parameters
        optimizer.step()
        optimizer.zero_grad()
        
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

***** Running training *****
  Num examples = 28000
  Batch size = 32
  Num steps = 4375


/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
Epoch:  20%|██        | 1/5 [17:33<1:10:15, 1053.75s/it]

Train loss: 0.2618107244712966


Epoch:  40%|████      | 2/5 [34:53<52:28, 1049.62s/it]  

...

Train loss: 0.14468977504542896


Epoch:  80%|████████  | 4/5 [1:09:18<17:20, 1040.82s/it]

Train loss: 0.11406514272093773


Epoch: 100%|██████████| 5/5 [1:26:31<00:00, 1038.28s/it]

Train loss: 0.08420646404155663


In [39]:
xlnet_out_address = 'models/ActorArtistAuthorPolitician/tc02'
# Make dir if not exits
if not os.path.exists(xlnet_out_address):
        os.makedirs(xlnet_out_address)

In [40]:
# Save a trained model, configuration and tokenizer
model_to_save = model.module if hasattr(model, 'module') else model  # Only save the model it-self

In [41]:
# If we save using the predefined names, we can load using `from_pretrained`
output_model_file = os.path.join(xlnet_out_address, "pytorch_model.bin")
output_config_file = os.path.join(xlnet_out_address, "config.json")

In [42]:
# Save model into file
torch.save(model_to_save.state_dict(), output_model_file)
model_to_save.config.to_json_file(output_config_file)
tokenizer.save_vocabulary(xlnet_out_address)

('models/ActorArtistAuthorPolitician/tc02/spiece.model',)

In [43]:
# Load model
model = XLNetForSequenceClassification.from_pretrained(xlnet_out_address,num_labels=len(tag2idx))

In [44]:
# Set model to GPU
model.to(device);
if n_gpu >1:
    model = torch.nn.DataParallel(model)

In [45]:
# Eval
# Evalue loop
model.eval();


In [46]:
# Set acc funtion
def accuracy(out, labels):
    outputs = np.argmax(out, axis=1)
    return np.sum(outputs == labels)

In [47]:
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(val_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(valid_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
#     print(tmp_eval_accuracy)
#     print(np.argmax(logits, axis=1))
#     print(label_ids)
    
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1
    
    
eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))

# Save the report into file
output_eval_file = os.path.join(xlnet_out_address, "eval_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =12000
  Batch size = 32


/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Eval results *****
  eval_accuracy = 0.93275
  eval_loss = 0.3193906268278758
  loss = 0.08420646404155663
             precision    recall  f1-score   support

          0       0.93      0.95      0.94      2968
          1       0.97      0.90      0.94      3773
          2       0.91      0.94      0.92      3034
          3       0.91      0.95      0.93      2225

avg / total       0.93      0.93      0.93     12000



In [48]:
#######Evaluating Training results
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0

y_true = []
y_predict = []
print("***** Running evaluation *****")
print("  Num examples ={}".format(len(tr_inputs)))
print("  Batch size = {}".format(batch_num))
for step, batch in enumerate(train_dataloader):
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_segs,b_labels = batch
    
    with torch.no_grad():
        outputs = model(input_ids =b_input_ids,token_type_ids=b_segs, input_mask = b_input_mask,labels=b_labels)
        tmp_eval_loss, logits = outputs[:2]
    
    # Get textclassification predict result
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()
    tmp_eval_accuracy = accuracy(logits, label_ids)
#     print(tmp_eval_accuracy)
#     print(np.argmax(logits, axis=1))
#     print(label_ids)
    
    # Save predict and real label reuslt for analyze
    for predict in np.argmax(logits, axis=1):
        y_predict.append(predict)
        
    for real_result in label_ids.tolist():
        y_true.append(real_result)

    
    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy
   
    nb_eval_steps += 1
    
    
eval_loss = eval_loss / nb_eval_steps
eval_accuracy = eval_accuracy / len(val_inputs)
loss = tr_loss/nb_tr_steps 
result = {'eval_loss': eval_loss,
                  'eval_accuracy': eval_accuracy,
                  'loss': loss}
report = classification_report(y_pred=np.array(y_predict),y_true=np.array(y_true))

# Save the report into file
output_eval_file = os.path.join(xlnet_out_address, "train_results.txt")
with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    for key in sorted(result.keys()):
        print("  %s = %s"%(key, str(result[key])))
        writer.write("%s = %s\n" % (key, str(result[key])))
        
    print(report)
    writer.write("\n\n")  
    writer.write(report)

***** Running evaluation *****
  Num examples =28000
  Batch size = 32


/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/functional.py:1374: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")
/usr/local/anaconda2/envs/dominantType/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


***** Eval results *****
  eval_accuracy = 2.30233333333
  eval_loss = 0.05020299577074391
  loss = 0.08420646404155663
             precision    recall  f1-score   support

          0       0.98      1.00      0.99      7032
          1       1.00      0.97      0.98      8307
          2       0.98      0.99      0.99      7336
          3       0.98      0.99      0.99      5325

avg / total       0.99      0.99      0.99     28000

